In [ ]:
# Importacion de librerias
import pygamer
print("PyGAMer version:", pygamer.__version__())
import numpy as np
import threevis
import trimesh
import time

In [ ]:
#Lectura del archivo de texto
with open("Lista_Moleculas_Pequeñas.txt","r") as f:
    lines = f.readlines()
for line in lines:
    line = line.split()
    LG.append(line[0])
print(LG)

In [ ]:
start = time.time()
f1=open("EsferaVerticesElementos.txt","w")
for i in range(len(LG)):
    file =LG[i]
    print(i,file)
    # Llama a las mallas de archivos off para generar el cascaron
    mesh01 = pygamer.readOFF(file+".off") #Malla interior
    mesh02 = pygamer.readOFF(file+".off") #Malla exterior
    
    #Caso especifico de mas de una malla
    mesh_split1 = mesh01.splitSurfaces()
    mesh_split2 = mesh02.splitSurfaces()
    print("Found %i meshes"%len(mesh_split1))
    print("Found %i meshes"%len(mesh_split2))
    mesh1 = mesh_split1[0] #Malla interior
    mesh2 = mesh_split2[0] #Malla interior
    
    # Corrige el vector normal de las mallas
    mesh1.correctNormals()
    mesh2.correctNormals()
    
    gInfo = mesh1.getRoot()
    gInfo.ishole = True
    # Informacion de la malla 2
    gInfo = mesh2.getRoot()
    gInfo.ishole = False
    
    # Construye una lista de mallas para la funcion TetGen
    meshes = [mesh1,mesh2]
    #Tetrahedriza la lista de mallas para generar el casacaron con TetGen.
    tetmesh = pygamer.makeTetMesh(meshes, "-pq1.1aYAO2/3")
    pygamer.writeDolfin('outputTetMesh'+file+'.xml', tetmesh)

    V1 = mesh1.nVertices
    E1 = mesh1.nFaces
    V2 = mesh2.nVertices
    E2 = mesh2.nFaces
    VC = tetmesh.nVertices
    
    f1.write(file+" "+str(V1)+" "+str(E1)+" "+str(V2)+" "+str(E2)+" "+str(VC)+"\n")  #Nombre VerticeR0 ElementosR0 VerticesR2 ElementosR2 #Vertices Cascaron 
f1.close()
end = time.time()
print(end - start)

In [ ]:
#Correccion de agujeros en la malla superficial
mesh3 = pygamer.readOFF(file+".off") #Malla con agujeros
mesh3.fillHoles()
mesh_split3 = mesh3.splitSurfaces()
print("Found %i meshes"%len(mesh_split3))
pygamer.writeOFF(file+".off", mesh3) #Malla sin agujeros